---   
# HW3 - Transfer learning

#### Due October 30, 2019

In this assignment you will learn about transfer learning. This technique is perhaps one of the most important techniques for industry. When a problem you want to solve does not have enough data, we use a different (larger) dataset to learn representations which can help us solve our task using the smaller task.

The general steps to transfer learning are as follows:

1. Find a huge dataset with similar characteristics to the problem you are interested in.
2. Choose a model powerful enough to extract meaningful representations from the huge dataset.
3. Train this model on the huge dataset.
4. Use this model to train on the smaller dataset.


### This homework has the following sections:
1. Question 1: MNIST fine-tuning (Parts A, B, C, D).
2. Question 2: Pretrain on Wikitext2 (Part A, B, C, D)
3. Question 3: Finetune on MNLI (Part A, B, C, D)
4. Question 4: Finetune using pretrained BERT (Part A, B, C)

---   
## Question 1 (MNIST transfer learning)
To grasp the high-level approach to transfer learning, let's first do a simple example using computer vision. 

The torchvision library has pretrained models (resnets, vggnets, etc) on the Imagenet dataset. Imagenet is a dataset
with 1.3 million images covering over 1000 classes of objects. When you use one of these models, the weights of the model initialize
with the weights saved from training on imagenet.

In this task we will:
1. Choose a pretrained model.
2. Freeze the model so that the weights don't change.
3. Fine-tune on a few labels of MNIST.   

#### Choose a model
Here we pick any of the models from torchvision

In [95]:
import torch
import torchvision.models as models

class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

# init the pretrained feature extractor
pretrained_resnet18 = models.resnet18(pretrained=True)

# we don't want the built in last layer, we're going to modify it ourselves
pretrained_resnet18.fc = Identity()

#### Freeze the model
Here we freeze the weights of the model. Freezing means the gradients will not backpropagate
into these weights.

By doing this you can think about the model as a feature extractor. This feature extractor outputs
a **representation** of an input. This representation is a matrix that encodes information about the input.

In [45]:
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False
        
def unfreeze_model(model):
    for param in model.parameters():
        param.requires_grad = True
        
freeze_model(pretrained_resnet50)

#### Init target dataset
Here we define the dataset we are actually interested in.

In [46]:
import os
from torchvision import transforms
from torchvision.datasets import  MNIST
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F

#  train/val  split
mnist_dataset = MNIST(os.getcwd(), train=True, download=True, transform=transforms.ToTensor())
mnist_train, mnist_val = random_split(mnist_dataset, [55000, 5000])

mnist_train = DataLoader(mnist_train, batch_size=32)
mnist_val = DataLoader(mnist_val, batch_size=32)

# test split
mnist_test = DataLoader(MNIST(os.getcwd(), train=False, download=True, transform=transforms.ToTensor()), batch_size=32)

### Part A (init fine-tune model)
decide what model to use for fine-tuning

In [94]:
def init_fine_tune_model():
    
    # YOUR CODE HERE
    
    return fine_tune_model

### Part B (Fine-tune (Frozen))

The actual problem we care about solving likely has a different number of classes or is a different task altogether. Fine-tuning is the process of using the extracted representations (features) to solve this downstream task  (the task you're interested in).

To illustrate this, we'll use our pretrained model (on Imagenet), to solve the MNIST classification task.

There are two types of finetuning. 

#### 1. Frozen feature_extractor
In the first type we pretrain with the FROZEN feature_extractor and NEVER unfreeze it during finetuning.


#### 2. Unfrozen feature_extractor
In the second, we finetune with a FROZEN feature_extractor for a few epochs, then unfreeze the feature extractor and finish training.


In this part we will use the first version

In [68]:
import torch.optim as optim

def FROZEN_fine_tune_mnist(feature_extractor, fine_tune_model, mnist_train, mnist_val):
    """
    model is a feature extractor (resnet).
    Create a new model which uses those features to finetune on MNIST
    
    return the fine_tune model
    """     
    
    # INSERT YOUR CODE: (train the fine_tune model using features extracted by feature_extractor)
    
    pass

### Part C (compute test accuracy)
Compute the test accuracy of fine-tuned model on MNIST

In [38]:
def calculate_mnist_test_accuracy(feature_extractor, fine_tune_model, mnist_test):
    
    test_accuracy = 0 # YOUR CODE HERE...
    
    return test_accuracy

### Grade!
Let's see how you did

In [97]:
def grade_mnist_frozen():
    
    # init a ft model
    fine_tune_model = init_fine_tune_model()
    
    # run the transfer learning routine
    FROZEN_fine_tune_mnist(pretrained_resnet18, fine_tune_model, mnist_train, mnist_val)
    
    # calculate test accuracy
    test_accuracy = calculate_mnist_test_accuracy(pretrained_resnet18, fine_tune_model, mnist_test)
    
    # the real threshold will be released by Oct 11 
    assert test_accuracy > 0.0, 'your accuracy is too low...'
    
    return test_accuracy
    
frozen_test_accuracy = grade_mnist_frozen()

### Part D (Fine-tune Unfrozen)
Now we'll learn how to train using the "unfrozen" approach.

In this approach we'll:
1. keep the feature_extract frozen for a few epochs (10)
2. Unfreeze it.
3. Finish training

In [71]:
def UNFROZEN_fine_tune_mnist(feature_extractor, fine_tune_model, mnist_train, mnist_val):
    """
    model is a feature extractor (resnet).
    Create a new model which uses those features to finetune on MNIST
    
    return the fine_tune model
    """     
    
    # INSERT YOUR CODE:
    # keep frozen for 10 epochs
    # ... train
    # unfreeze
    # train for rest of the time
    
    pass

### Grade UNFROZEN
Let's see if there's a difference in accuracy!

In [73]:
def grade_mnist_unfrozen():
    
    # init a ft model
    fine_tune_model = init_fine_tune_model()
    
    # run the transfer learning routine
    FROZEN_fine_tune_mnist(pretrained_resnet50, fine_tune_model, mnist_train, mnist_val)
    
    # calculate test accuracy
    test_accuracy = calculate_mnist_test_accuracy(pretrained_resnet50, fine_tune_model, mnist_test)
    
    # the real threshold will be released by Oct 11 
    assert test_accuracy > 0.0, 'your accuracy is too low...'
    
    return test_accuracy
    
unfrozen_test_accuracy = grade_mnist_unfrozen()

In [75]:
assert unfrozen_test_accuracy > frozen_test_accuracy, 'the unfrozen model should be better'

--- 
# Question 2 (train a model on Wikitext-2)

Here we'll apply what we just learned to NLP. In this section we'll make our own feature extractor and pretrain it on Wikitext-2.

The WikiText language modeling dataset is a collection of over 100 million tokens extracted from the set of verified Good and Featured articles on Wikipedia. The dataset is available under the Creative Commons Attribution-ShareAlike License.

#### Part A
In this section you need to generate the training, validation and test split. Feel free to use code from your previous lectures.

In [76]:
from torchtext.datasets import WikiText2


def init_wikitext_dataset():
    """
    Fill in the details
    """
    wikitext_val = None # YOUR CODE HERE
    wikitext_train = None # YOUR CODE HERE
    wikitext_test = None # YOUR CODE HERE
    
    return wikitext_train, wikitext_val, wikitext_test

#### Part B   
Here we design our own feature extractor. In MNIST that was a resnet because we were dealing with images. Now we need to pick a model that can model sequences better. Design an RNN-based model here.

In [77]:
def init_feature_extractor():
    
    feature_extractor = None #  YOUR CODE
    
    return feature_extractor

#### Part C
Pretrain the feature extractor

In [79]:
def fit_feature_extractor(feature_extractor, wikitext_train, wikitext_val):
    # FILL IN THE DETAILS
    pass

#### Part D
Calculate the test perplexity on wikitext2. Feel free to recycle code from previous assignments from this class. 

In [78]:
def calculate_wiki2_test_perplexity(feature_extractor, wikitext_test):
    
    # FILL IN DETAILS
    
    return test_ppl

#### Let's grade your results!
(don't touch this part)

In [28]:
def grade_wikitext2():
    # load data
    wikitext_train, wikitext_val, wikitext_test = init_wikitext_dataset()

    # load feature extractor
    feature_extractor = init_feature_extractor()

    # pretrain using the feature extractor
    fit_feature_extractor(feature_extractor, wikitext_train, wikitext_val)

    # check test accuracy
    test_ppl = calculate_wiki2_test_perplexity(feature_extractor, wikitext_test)

    # the real threshold will be released by Oct 11 
    assert test_ppl < 10000, 'ummm... your perplexity is too high...'
    
grade_wikitext2()

---   
## Question 3 (fine-tune on MNLI)
In this question you will use your feature_extractor from question 2
to fine-tune on MNLI.

(From the website):
The Multi-Genre Natural Language Inference (MultiNLI) corpus is a crowd-sourced collection of 433k sentence pairs annotated with textual entailment information. The corpus is modeled on the SNLI corpus, but differs in that covers a range of genres of spoken and written text, and supports a distinctive cross-genre generalization evaluation. The corpus served as the basis for the shared task of the RepEval 2017 Workshop at EMNLP in Copenhagen.

MNLI has 3 genres (3 classes).
The goal of this question is to maximize the test accuracy in MNLI. 

### Part A
In this section you need to generate the training, validation and test split. Feel free to use code from your previous lectures.

In [80]:
from torchtext.datasets import MultiNLI

def init_mnli_dataset():
    """
    Fill in the details
    """
    mnli_val = None # TODO
    mnli_train = None # TODO
    mnli_test = None # TODO
    
    return mnli_train, mnli_val, mnli_test

### Part B
Here we again design a model for finetuning. Use the output of your feature-extractor as the input to this model. This should be a powerful classifier (up to you).

In [81]:
def init_finetune_model():
    
    # TODO FILL IN THE DETAILS
    fine_tune_model = ...
    
    return fine_tune_model

### Part C
Use the feature_extractor and your fine_tune_model to fine_tune MNLI

In [82]:
def fine_tune_mnli(feature_extractor, fine_tune_model, mnli_train, mnli_val):
    # YOUR CODE HERE
    pass

### Part D
Evaluate the test accuracy

In [83]:
def calculate_mnli_test_accuracy(feature_extractor, fine_tune_model, mnli_test):
    
    # YOUR CODE HERE...
    
    return test_ppl

### Let's grade your results

In [55]:
def grade_mnli():
    # load data
    mnli_train, mnli_val, mnli_test = init_mnli_dataset()

    # no need to load feature extractor because it is fine-tuned
    feature_extractor = feature_extractor

    # init the fine_tune model
    fine_tune_model = init_finetune_model()
    
    # finetune
    fine_tune_mnli(feature_extractor, fine_tune_model, mnli_train, mnli_val)

    # check test accuracy
    test_accuracy = calculate_mnli_test_accuracy(feature_extractor, wikitext_test)

    # the real threshold will be released by Oct 11 
    assert test_ppl > 0.00, 'ummm... your accuracy is too low...'
    
grade_mnli()

---  
### Question 4 (BERT)

A major direction in research came from a model called BERT, released last year.  

In this question you'll use BERT as your feature_extractor instead of the model you
designed yourself.

To get BERT, head on over to (https://github.com/huggingface/transformers) and load your BERT model here

In [ ]:
!pip install transformers

### Part A (init BERT)
In this section you need to create an instance of BERT and return if from the function

In [87]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

def init_bert():
    
    BERT = None # ... YOUR CODE HERE
    
    return BERT

## Part B (fine-tune with BERT)

Use BERT as your feature extractor to finetune MNLI. Use a new finetune model (reset weights).

In [88]:
def fine_tune_mnli_BERT(BERT_feature_extractor, fine_tune_model, mnli_train, mnli_val):
    # YOUR CODE HERE
    pass

## Part C
Evaluate how well we did

In [90]:
def calculate_mnli_test_accuracy_BERT(feature_extractor, fine_tune_model, mnli_test):
    
    # YOUR CODE HERE...
    
    return test_ppl

## Let's grade your BERT results!

In [91]:
def grade_mnli_BERT():
    BERT_feature_extractor = init_bert()
    
    # load data
    mnli_train, mnli_val, mnli_test = init_mnli_dataset()

    # init the fine_tune model
    fine_tune_model = init_finetune_model()
    
    # finetune
    fine_tune_mnli(BERT_feature_extractor, fine_tune_model, mnli_train, mnli_val)

    # check test accuracy
    test_accuracy = calculate_mnli_test_accuracy(feature_extractor, wikitext_test)
    
    # the real threshold will be released by Oct 11 
    assert test_ppl > 0.0, 'ummm... your accuracy is too low...'
    
grade_mnli_BERT()